# Long short term memory

We previously explored RNNs, neural networks that are able to propagate some hidden state through a rolled out version of itself. A major problem with RNNs is exploding or vanishing gradients. Gradient clipping solves the exploding gradient problem, but the vanishing gradient problem is harder to solve. LSTMs propose a different architecture which benefits from a hidden state like RNNs, but mitigates the vanishing gradient problem. Another issue RNNs have is that the hidden state often forgets information from a while ago in the sequence and is more biased towards more recent tokens. LSTMs also address this issue with their gated structure. 


In [64]:
from torch import nn
import torch 
from torch.nn.utils.rnn import PackedSequence
import torch.nn.functional
from torch.utils.data import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

## LSTM model defintion
The goal here is to build a substitute for the torch.nn.rnn.lstm module. This should be able to handle packed sequences and batched data the same way the source code for that module does. For now, does not need to have multiple layers or be bidirectional.

In [65]:
class lstm(nn.Module): 
    def __init__(self, input_size, hidden_dim, output_dim=1) -> None:
        super().__init__()
        self.input_dim = input_size
        self.hidden_dim  = hidden_dim
        
        self.forget_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.input_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.input_node = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Tanh()
        )
        self.output_gate = nn.Sequential(
            nn.Linear(input_size+hidden_dim, hidden_dim),
            nn.Sigmoid()
        )
        self.tanh = nn.Tanh()

        # this output layer can be fancier if needed by the use case
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h_in=None, c_in=None):

        if isinstance(x, PackedSequence):
            input, batch_sizes, sorted_indices, unsorted_indices = x
            max_batch_size = batch_sizes[0]
            if h_in is None: 
                h_in = self.init_h(max_batch_size, x)
                c_in = self.init_h(max_batch_size, x)
        
        data_offset = 0
        outputs = []
        for batch_size in batch_sizes:
            current_input = input[data_offset:data_offset + batch_size]
            data_offset += batch_size
            active_h = h_in[:,:batch_size,:]
            active_c = c_in[:,:batch_size,:]
            combined = torch.cat([current_input.unsqueeze(0), active_h], dim=2)
            
            i_gate_output = self.input_gate(combined)
            i_node_output = self.input_node(combined)
            o_gate_output = self.output_gate(combined)
            f_gate_output = self.forget_gate(combined)

            c_out = (f_gate_output * active_c) + (i_node_output * i_gate_output)

            h_out = self.tanh(c_out) * o_gate_output
            out = self.output_layer(h_out)
            h_in[:,:batch_size,:] = h_out
            c_in[:,:batch_size,:] = c_out
            outputs.append(out)

            # Handle decreasing batch size
            h_in[:,batch_size:,:] = 0
            c_in[:,batch_size:,:] = 0

       
        if isinstance(x, PackedSequence):
            output_packed = PackedSequence(outputs, batch_sizes, sorted_indices, unsorted_indices)
            return output_packed, h_out, c_out
        
        return out, h_out, c_out
    
    
    def init_h(self, batch_size, x):
        #alternatives include but not limited to Xavier/Kaiminh initialization
        return torch.zeros(1, batch_size, self.hidden_dim, dtype=x.data.dtype, device=x.data.device)

In [66]:
class newsLSTM(nn.Module): 
    def __init__(self, vocab_size, embed_size, hidden_size) -> None:
        super(newsLSTM, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.hidden_size = hidden_size 
        
        self.lstm = lstm(input_size=embed_size, hidden_dim=hidden_size)
        
        self.hidden2label = nn.Linear(2*hidden_size, 4)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropoutLayer = nn.Dropout(p=0.5)

    def forward(self, x, x_len):
        embedded = self.encoder(x)
        x_packed = nn.utils.rnn.pack_padded_sequence(embedded, x_len, batch_first=True, enforce_sorted=False)
        output, h_t, c_t = self.lstm(x_packed)  # Pass the initial hidden state 'h' to the RNN
        print(h_t.shape)
        
        hidden = self.dropoutLayer(torch.cat((h_t[-2,:,:], h_t[-1,:,:]), dim=1))
        
        # Linear layer and softmax
        label_space = self.hidden2label(hidden)
        
        return label_space
    

We repeat the exact same news classification task performed by in the bidirectionalRNN subdir

In [67]:
train_iter = AG_NEWS(split='train')

# Convert to list to enable random splitting
train_dataset = list(train_iter)

#80-20 train-val split 
train_size = int(len(train_dataset) * 0.8)  
val_size = len(train_dataset) - train_size  
train_data, val_data = random_split(train_dataset, [train_size, val_size])

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

VOCAB_SIZE = 5000

# Build vocab based on the train_data
train_data_iter = (text for _, text in train_data)
vocab = build_vocab_from_iterator(yield_tokens(train_data_iter), specials=["<unk>"], max_tokens=VOCAB_SIZE)
vocab.set_default_index(vocab["<unk>"])

In [68]:
train_data[0]

(3,
 'Wrigley swallows \\$1.5bn Life Savers Kraft, the struggling American food group, yesterday announced the \\$1.5bn (810m) sale of its Life Savers and Altoids confectionery division to the chewing gum business Wm Wrigley Jr.')

In [69]:
vocab(['word', 'probably', 'unknown', 'gibberish'])

[1961, 1643, 4837, 0]

In [70]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [71]:
vocab.lookup_tokens([4999])

['\\$9']

In [72]:
def collate_batch(batch):
    device = torch.device("cpu")
    label_list, text_list, lengths = [], [], []
    
    # Sort the batch in the descending order
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    lengths = torch.tensor(lengths, dtype=torch.int64)
    
    # Pad sequences
    text_list = pad_sequence(text_list, batch_first=True)
    
    return label_list.to(device), text_list.to(device), lengths

In [73]:
train_loader = DataLoader(train_data, batch_size = 8, shuffle = True, collate_fn = collate_batch)
val_loader = DataLoader(val_data, batch_size = 8, shuffle = False, collate_fn = collate_batch)

In [74]:
batch = next(iter(train_loader))

# Inspect the shape of the input data
input_data = batch[1]  # Assuming the input data is the first element of the batch
input_shape = input_data.shape[0]

In [75]:
batch

(tensor([0, 1, 3, 3, 2, 0, 0, 3]),
 tensor([[ 881,   20,  363, 1931, 2490,  923,  779, 2114,    3,  363,   15,    5,
           892,    0,    0,    5, 1165,  397, 1668,    5, 3628, 1931,  995, 2490,
             7,   30, 1417, 1563,  147,  227,   64,    0,    3,  460,   20,  200,
           779,  102,    8, 1850, 1808,    3,  148,   26,    1,  193,   26,   17,
          1000,    6,  102,   79, 1668,    2,    0, 2490,    7,    2,  147,    6,
             0,   20,    2,   99,    6,    2,  881,    3,  103,    0, 4269,    0,
             8,  287, 1080, 1567,    0, 1668,    1,    1,    1],
         [3270,  398, 2780,   11,    0,  493, 4790, 3270,    3,   70,    7,    2,
          1861,  534,    8,   46,    7,    2, 1860, 1317,  177,  333,    3,    0,
            59, 4002,  164, 2897,   64,    8,  874,  187,    2, 2780,   11,    2,
             0, 1197, 1520,    3,  103,   33, 3719,    2,  141,  535,    1,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0

In [76]:
batch[1].shape

torch.Size([8, 81])

In [77]:
len(batch)

3

In [78]:
a =  torch.ones(5, 50)
a =  torch.ones(5, 50)
a =  torch.ones(5, 50)

In [79]:
input_shape

8

In [80]:
print(batch[0])
print(batch[1].shape)

tensor([0, 1, 3, 3, 2, 0, 0, 3])
torch.Size([8, 81])


In [81]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
NUM_EPOCHS = 50
DROPOUT = 0.5
DEVICE = torch.device('cpu')

EMBEDDING_DIM = 128
BIDIRECTIONAL = True
HIDDEN_DIM = 128
NUM_LAYERS = 2
OUTPUT_DIM = 4

In [82]:
model = newsLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [83]:
def train(model, train_loader, val_loader, loss_function, optim, epochs, device):
    losses = [] #group losses for loss visualization 
    running_loss = 0.0
    for epoch in range(epochs):
        model.train()
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)
    
        for i, batch_data in enumerate(train_loader):
            
            model.train()
            (y, x, x_size) = batch_data
            #print("Labels: {}, data: {}, x_size.cpu(): {}".format(batch_data[0], x.shape,x_size.cpu()))

            logits = model(x, x_size.cpu())
            #print("Target size: {}, pred_size: {}".format(y.size(), logits.size()))
            loss = loss_function(logits, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            running_loss += loss.item()
            losses.append(loss)

            if (i+1) % 1000 == 0:
                print("Step: {}, average training loss over last 2000 steps: {:.4f}".format(i+1, running_loss/1000))
                running_loss = 0.0
            
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for i, batch_data in enumerate(val_loader):
                (y, x, x_size) = batch_data
                y, x, x_size = y.to(device), x.to(device), x_size.to(device)
                
                logits = model(x, x_size.cpu())
                loss = loss_function(logits, y)
                
                val_loss += loss.item()
        
        print("Epoch: {}, validation loss: {:.4f}".format(epoch+1, val_loss/len(val_loader)))

In [84]:
train(model, train_loader, val_loader, torch.nn.functional.cross_entropy, optimizer, NUM_EPOCHS, DEVICE)

Epoch 1 / 50
----------
original input shape: torch.Size([371, 128])
batch_size: 8
hidden_shape = torch.Size([1, 8, 128])
current input size = torch.Size([8, 128])
combined shape = torch.Size([1, 8, 256])
<class 'torch.Tensor'>
batch size: 8, h_in size: torch.Size([1, 8, 128]), h_out shape: torch.Size([1, 8, 128])
original input shape: torch.Size([371, 128])
batch_size: 8
hidden_shape = torch.Size([1, 8, 128])
current input size = torch.Size([8, 128])
combined shape = torch.Size([1, 8, 256])
<class 'torch.Tensor'>
batch size: 8, h_in size: torch.Size([1, 8, 128]), h_out shape: torch.Size([1, 8, 128])
original input shape: torch.Size([371, 128])
batch_size: 8
hidden_shape = torch.Size([1, 8, 128])
current input size = torch.Size([8, 128])
combined shape = torch.Size([1, 8, 256])
<class 'torch.Tensor'>
batch size: 8, h_in size: torch.Size([1, 8, 128]), h_out shape: torch.Size([1, 8, 128])
original input shape: torch.Size([371, 128])
batch_size: 8
hidden_shape = torch.Size([1, 8, 128])
cu

IndexError: index -2 is out of bounds for dimension 0 with size 1